### **CSV to SQLite Converter**
This script converts multiple CSV files into a single SQLite database, creating one table per CSV file.

#### **How It Works**
1. **Scans CSV folder** - Loops through all `.csv` files in the specified directory
2. **Creates tables** - Each CSV becomes a table (filename = table name)
3. **Auto-detects columns** - Reads the first row to generate column names
4. **Imports data** - Loads all CSV data into the corresponding tables
5. **Single database output** - All tables are stored in one `.db` file

#### **Key Features**
- ✅ Automatic column detection from CSV headers
- ✅ Data type inference (INTEGER, REAL, TEXT)
- ✅ Handles empty values (converts to NULL)
- ✅ Batch processing with progress tracking
- ✅ Database verification with statistics

#### **Configuration**
```python
CSV_FILES_FOLDER = "../data/csv_db/"      # Input folder
DB_FILE = "../data/databases/combined.db" # Output database
```

#### **Usage**
Simply run the script - it automatically processes all CSV files in the input folder and generates a single SQLite database.

In [4]:
import sqlite3
import csv
import os

# Configuration
CSV_FILES_FOLDER = "../data/csv_db/"
DB_FILE = "../data/databases/movie.db"

def clean_column_name(name):
    """Nettoie le nom de colonne pour SQLite"""
    return name.strip().replace(' ', '_').replace('-', '_').lower()

def clean_data(value):
    """Nettoie les valeurs du CSV"""
    return None if value == "" else value

def get_sql_type(value):
    """Détermine le type SQL basique"""
    if value is None or value == "":
        return "TEXT"
    try:
        int(value)
        return "INTEGER"
    except:
        try:
            float(value)
            return "REAL"
        except:
            return "TEXT"

def create_table_from_csv(csv_file, table_name, conn):
    """Crée une table basée sur la structure du CSV"""
    cursor = conn.cursor()
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        headers = next(csv_reader)
        first_row = next(csv_reader, None)
        
        # Nettoyer les noms de colonnes
        clean_headers = [clean_column_name(h) for h in headers]
        
        # Créer la définition de la table
        columns_def = []
        for i, col in enumerate(clean_headers):
            sql_type = get_sql_type(first_row[i] if first_row else None)
            columns_def.append(f"{col} {sql_type}")
        
        # Créer la table
        create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns_def)})"
        cursor.execute(create_query)
        print(f"✅ Table '{table_name}' créée avec {len(clean_headers)} colonnes")
    
    conn.commit()
    return clean_headers

def import_csv_to_table(csv_file, table_name, headers, conn):
    """Importe les données du CSV dans la table"""
    cursor = conn.cursor()
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        
        placeholders = ', '.join(['?' for _ in headers])
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        
        row_count = 0
        for row in csv_reader:
            values = [clean_data(row[original_header]) for original_header in csv_reader.fieldnames]
            cursor.execute(insert_query, values)
            row_count += 1
            
            if row_count % 1000 == 0:
                conn.commit()
        
        conn.commit()
        print(f"✅ {row_count} lignes importées dans '{table_name}'")

def verify_database(conn):
    """Vérifie le contenu de la base de données"""
    cursor = conn.cursor()
    
    # Lister toutes les tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    print(f"\n{'='*50}")
    print(f"📊 STATISTIQUES DE LA BASE DE DONNÉES")
    print(f"{'='*50}")
    print(f"Nombre de tables: {len(tables)}\n")
    
    for (table_name,) in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"📋 Table '{table_name}': {count} lignes")

# Programme principal
conn = sqlite3.connect(DB_FILE)
print(f"🔄 Création de la base de données: {DB_FILE}\n")

for filename in os.listdir(CSV_FILES_FOLDER):
    if filename.endswith(".csv"):
        csv_file = os.path.join(CSV_FILES_FOLDER, filename)
        table_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_").lower()
        
        print(f"\n{'='*50}")
        print(f"🔄 Traitement de: {filename}")
        
        headers = create_table_from_csv(csv_file, table_name, conn)
        import_csv_to_table(csv_file, table_name, headers, conn)

verify_database(conn)
conn.close()

print(f"\n✅ Base de données complète créée: {DB_FILE}")

🔄 Création de la base de données: ../data/databases/movie.db


🔄 Traitement de: amazon_prime_titles.csv
✅ Table 'amazon_prime_titles' créée avec 12 colonnes
✅ 9668 lignes importées dans 'amazon_prime_titles'

🔄 Traitement de: disney_plus_titles.csv
✅ Table 'disney_plus_titles' créée avec 12 colonnes
✅ 1450 lignes importées dans 'disney_plus_titles'

🔄 Traitement de: netflix_titles.csv
✅ Table 'netflix_titles' créée avec 12 colonnes
✅ 8807 lignes importées dans 'netflix_titles'

📊 STATISTIQUES DE LA BASE DE DONNÉES
Nombre de tables: 3

📋 Table 'amazon_prime_titles': 38672 lignes
📋 Table 'disney_plus_titles': 5800 lignes
📋 Table 'netflix_titles': 35228 lignes

✅ Base de données complète créée: ../data/databases/movie.db


### **OMDB API Documentation Generator**

Creates a structured JSON documentation of the OMDB API (Open Movie Database) to enrich movie data. This JSON file can be fed to LLMs for automated API interactions.

#### **What It Does**

1. **Defines API structure** in a Python dictionary
2. **Exports to JSON** for easy consumption by LLMs or other tools
3. **Documents all parameters** and response fields

#### **Key Features**

- ✅ API key requirement clearly marked
- ✅ All parameters documented with examples
- ✅ Response structure fully detailed
- ✅ LLM-friendly format for automated requests
- ✅ Human-readable JSON output

#### **Usage**

Run the script to generate `omdb_api_doc.json` - this file can be:
- Fed to LLMs for API automation
- Used as reference documentation
- Integrated into data enrichment pipelines

#### **API Access**

Get your free API key at: http://www.omdbapi.com/apikey.aspx

In [ ]:
import sqlite3
import csv
import os

# Configuration
CSV_FILES_FOLDER = "../data/csv_db/"
DB_FILE = "../data/databases/movie_2.db"

def clean_column_name(name):
    """Nettoie le nom de colonne pour SQLite"""
    return name.strip().replace(' ', '_').replace('-', '_').lower()

def clean_data(value):
    """Nettoie les valeurs du CSV"""
    return None if value == "" else value

def get_sql_type(value):
    """Détermine le type SQL basique"""
    if value is None or value == "":
        return "TEXT"
    try:
        int(value)
        return "INTEGER"
    except:
        try:
            float(value)
            return "REAL"
        except:
            return "TEXT"

def create_table_from_csv(csv_file, table_name, conn):
    """Crée une table basée sur la structure du CSV avec ID préfixé"""
    cursor = conn.cursor()
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        headers = next(csv_reader)
        first_row = next(csv_reader, None)
        
        # Nettoyer les noms de colonnes
        clean_headers = [clean_column_name(h) for h in headers]
        
        # Créer la définition de la table avec ID unique préfixé
        columns_def = [f"unique_id TEXT PRIMARY KEY"]  # ID unique pour toute la DB
        
        for i, col in enumerate(clean_headers):
            sql_type = get_sql_type(first_row[i] if first_row else None)
            columns_def.append(f"{col} {sql_type}")
        
        # Créer la table
        create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns_def)})"
        cursor.execute(create_query)
        print(f"✅ Table '{table_name}' créée avec {len(clean_headers) + 1} colonnes (dont unique_id)")
    
    conn.commit()
    return clean_headers

def import_csv_to_table(csv_file, table_name, headers, conn):
    """Importe les données du CSV dans la table avec ID préfixé"""
    cursor = conn.cursor()
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        
        # Ajouter unique_id dans les placeholders
        placeholders = ', '.join(['?' for _ in range(len(headers) + 1)])
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        
        row_count = 0
        for row in csv_reader:
            # Créer l'ID unique : table_name + "_" + row_number
            unique_id = f"{table_name}_{row_count + 1}"
            
            # Extraire les valeurs du CSV
            values = [clean_data(row[original_header]) for original_header in csv_reader.fieldnames]
            
            # Insérer avec unique_id en première position
            cursor.execute(insert_query, [unique_id] + values)
            row_count += 1
            
            if row_count % 1000 == 0:
                conn.commit()
                print(f"  Importé {row_count} lignes...")
        
        conn.commit()
        print(f"✅ {row_count} lignes importées dans '{table_name}'")

def verify_database(conn):
    """Vérifie le contenu de la base de données"""
    cursor = conn.cursor()
    
    # Lister toutes les tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    print(f"\n{'='*50}")
    print(f"📊 STATISTIQUES DE LA BASE DE DONNÉES")
    print(f"{'='*50}")
    print(f"Nombre de tables: {len(tables)}\n")
    
    for (table_name,) in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        
        # Afficher un exemple d'ID
        cursor.execute(f"SELECT unique_id FROM {table_name} LIMIT 1")
        sample_id = cursor.fetchone()
        
        print(f"📋 Table '{table_name}': {count} lignes")
        if sample_id:
            print(f"   Exemple d'ID: {sample_id[0]}")

# Programme principal
conn = sqlite3.connect(DB_FILE)
print(f"🔄 Création de la base de données: {DB_FILE}\n")

for filename in os.listdir(CSV_FILES_FOLDER):
    if filename.endswith(".csv"):
        csv_file = os.path.join(CSV_FILES_FOLDER, filename)
        table_name = filename.replace(".csv", "").replace("-", "_").replace(" ", "_").lower()
        
        print(f"\n{'='*50}")
        print(f"🔄 Traitement de: {filename}")
        print(f"   Nom de table: {table_name}")
        print(f"{'='*50}")
        
        headers = create_table_from_csv(csv_file, table_name, conn)
        import_csv_to_table(csv_file, table_name, headers, conn)

verify_database(conn)
conn.close()

print(f"\n✅ Base de données complète créée: {DB_FILE}")

In [ ]:
import json
from pathlib import Path

# ---- 1. The whole documentation as a Python dict (valid JSON) ----
omdb_doc = {
    "description": (
        "This documentation outlines the structure and specifications of the OMDB API "
        "(Open Movie Database). It is organized into two main dictionaries:\n"
        '- "arguments": Defines all input parameters accepted by the API, split into two query modes:\n'
        "    • by_id_or_title: Search by IMDb ID or exact title.\n"
        "    • by_search: Keyword-based search (fuzzy matching).\n"
        "  Each parameter includes: required status, data type, valid values, default value (if any), "
        "clear description, and example.\n"
        "  Note: In by_id_or_title mode, **at least one of 'i' or 't' is required**.\n"
        '- "responses": Describes the structure of the API\'s returned data, with data type '
        "and example for each field.\n"
        "These dictionaries enable building valid OMDB API requests and automatically parsing responses.\n\n"
        "**Important**: An API key (`apikey`) is **required** for all requests."
    ),
    "base_url": "http://www.omdbapi.com/",
    "arguments": {
        "by_id_or_title": {
            "apikey": {
                "required": True,
                "type": "str",
                "description": "API key required to access the service. Get one at http://www.omdbapi.com/apikey.aspx",
                "example": "apikey=12345678"
            },
            "i": {
                "required": False,
                "type": "str",
                "description": "IMDb ID of the movie, series, or episode (format: 'tt' followed by 7 or 8 digits). *At least 'i' or 't' is required.*",
                "example": "i=tt1285016"
            },
            "t": {
                "required": False,
                "type": "str",
                "description": "Exact title of the movie, series, or episode. *At least 'i' or 't' is required.*",
                "example": "t=Inception"
            },
            "type": {
                "required": False,
                "type": "str",
                "valid_options": ["movie", "series", "episode"],
                "default": None,
                "description": "Type of result to return (movie, series, or episode).",
                "example": "type=movie"
            },
            "y": {
                "required": False,
                "type": "str",
                "description": "Year of release (4-digit year).",
                "example": "y=2010"
            },
            "plot": {
                "required": False,
                "type": "str",
                "valid_options": ["short", "full"],
                "default": "short",
                "description": "Return a short or full plot summary.",
                "example": "plot=full"
            },
            "r": {
                "required": False,
                "type": "str",
                "valid_options": ["json", "xml"],
                "default": "json",
                "description": "Response data format.",
                "example": "r=json"
            },
            "callback": {
                "required": False,
                "type": "str",
                "description": "Callback function name for JSONP support.",
                "example": "callback=myFunction"
            },
            "v": {
                "required": False,
                "type": "str",
                "default": "1",
                "description": "API version (reserved for future use).",
                "example": "v=1"
            }
        },
        "by_search": {
            "apikey": {
                "required": True,
                "type": "str",
                "description": "API key required to access the service. Get one at http://www.omdbapi.com/apikey.aspx",
                "example": "apikey=12345678"
            },
            "s": {
                "required": True,
                "type": "str",
                "description": "Movie/series title to search for (keyword-based search).",
                "example": "s=Joker"
            },
            "type": {
                "required": False,
                "type": "str",
                "valid_options": ["movie", "series", "episode"],
                "default": None,
                "description": "Type of result to return (movie, series, or episode).",
                "example": "type=movie"
            },
            "y": {
                "required": False,
                "type": "str",
                "description": "Year of release (4-digit year).",
                "example": "y=2019"
            },
            "r": {
                "required": False,
                "type": "str",
                "valid_options": ["json", "xml"],
                "default": "json",
                "description": "Response data format.",
                "example": "r=json"
            },
            "page": {
                "required": False,
                "type": "int",
                "valid_options": list(range(1, 101)),
                "default": 1,
                "description": "Page number to return (1–100). Maximum 100 results per page.",
                "example": "page=2"
            },
            "callback": {
                "required": False,
                "type": "str",
                "description": "Callback function name for JSONP support.",
                "example": "callback=myFunction"
            },
            "v": {
                "required": False,
                "type": "str",
                "default": "1",
                "description": "API version (reserved for future use).",
                "example": "v=1"
            }
        }
    },
    "responses": {
        "Title": {"type": "str", "description": "Title of the movie or series.", "example": "Inception"},
        "Year": {"type": "str", "description": "Year of release (4-digit string).", "example": "2010"},
        "Rated": {"type": "str", "description": "MPAA rating (e.g., 'PG-13', 'R') or 'N/A'.", "example": "PG-13"},
        "Released": {"type": "str", "description": "Full release date (formatted as 'DD Mon YYYY').", "example": "16 Jul 2010"},
        "Runtime": {"type": "str", "description": "Runtime in minutes (with ' min' suffix).", "example": "148 min"},
        "Genre": {"type": "str", "description": "Comma-separated list of genres.", "example": "Action, Sci-Fi, Thriller"},
        "Director": {"type": "str", "description": "Comma-separated list of directors. 'N/A' if unknown.", "example": "Christopher Nolan"},
        "Writer": {"type": "str", "description": "Comma-separated list of writers. 'N/A' if unknown.", "example": "Christopher Nolan"},
        "Actors": {"type": "str", "description": "Comma-separated list of lead actors. 'N/A' if unknown.", "example": "Leonardo DiCaprio, Ken Watanabe, Joseph Gordon-Levitt"},
        "Plot": {"type": "str", "description": "Plot summary. Length depends on `plot=short` (default) or `plot=full`.", "example": "A thief who steals corporate secrets through dream-sharing technology..."},
        "Language": {"type": "str", "description": "Comma-separated list of languages.", "example": "English, Japanese, French"},
        "Country": {"type": "str", "description": "Comma-separated list of countries of origin.", "example": "USA, UK"},
        "Awards": {"type": "str", "description": "Awards won/nominated. 'N/A' if none.", "example": "Won 4 Oscars. Another 152 wins & 204 nominations."},
        "Poster": {"type": "str", "description": "URL to the poster image. 'N/A' if not available.", "example": "https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg"},
        "Ratings": {"type": "list[dict]", "description": "List of ratings from various sources. Each dict has 'Source' and 'Value'.", "example": [
            {"Source": "Internet Movie Database", "Value": "8.8/10"},
            {"Source": "Rotten Tomatoes", "Value": "87%"},
            {"Source": "Metacritic", "Value": "74/100"}
        ]},
        "Metascore": {"type": "str", "description": "Metacritic score (0–100). 'N/A' if not available.", "example": "74"},
        "imdbRating": {"type": "str", "description": "IMDb rating out of 10.", "example": "8.8"},
        "imdbVotes": {"type": "str", "description": "Number of votes on IMDb (with commas).", "example": "2,345,678"},
        "imdbID": {"type": "str", "description": "Unique IMDb ID (starts with 'tt').", "example": "tt1375666"},
        "Type": {"type": "str", "description": "Content type: 'movie', 'series', or 'episode'.", "example": "movie"},
        "DVD": {"type": "str", "description": "DVD release date ('DD Mon YYYY') or 'N/A'.", "example": "07 Dec 2010"},
        "BoxOffice": {"type": "str", "description": "Box office earnings (formatted with '$'). 'N/A' if unknown.", "example": "$292,576,195"},
        "Production": {"type": "str", "description": "Production company. 'N/A' if unknown.", "example": "Warner Bros., Legendary Entertainment"},
        "Website": {"type": "str", "description": "Official website URL. 'N/A' if none.", "example": "http://inceptionmovie.warnerbros.com/" },
        "Response": {"type": "str", "description": "Indicates if request was successful: 'True' or 'False'.", "example": "True"}
    }
}

# ---- 2. Write a *real* JSON file (pretty-printed) ----
path = Path("doc\\omdb_api_doc.json")
path.write_text(json.dumps(omdb_doc, indent=4), encoding="utf-8")
print(f"File written to: {path.resolve()}")

# ---- 3. Load it back (this will now work) ----
with path.open("r", encoding="utf-8") as f:
    loaded = json.load(f)

print("\nLoaded successfully!")
print("Title example:", loaded["responses"]["Title"]["example"])

File written to: C:\Users\Vincent\GitHub\Vincent-20-100\Agentic_Systems_Project_Vlamy\code\omdb_api_doc.json

Loaded successfully!
Title example: Inception
